# Introduce Gradio from HuggingFace

In [25]:
#Import libraries
import os
import requests

from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI 

import anthropic

import datetime

print(f"This notebook was last run at: {datetime.datetime.now():%Y-%m-%d %H:%M:%S}")

This notebook was last run at: 2025-09-22 20:45:33


In [2]:
import gradio as gr

In [3]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

OpenAI API Key exists and begins sk-proj-


In [4]:
openai = OpenAI()
system_message =  "You are a helpful, smart, kind assistant"

def message_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    completion = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
    )
    return completion.choices[0].message.content

# This can reveal the "training cut off", or the most recent date in the training data

message_gpt("What is today's date?")

"Today's date is October 1, 2023."

In [5]:
def shout(text):
    print(f"Shout has been called with input {text}")
    return text.upper()

shout("Hello")

# gr.Interface(fn=shout, inputs="textbox", outputs="textbox").launch()

Shout has been called with input Hello


'HELLO'

In [6]:
# Adding share=True means that it can be accessed publically
# A more permanent hosting is available using a platform called Spaces from HuggingFace, which we will touch on next week
# NOTE: Some Anti-virus software and Corporate Firewalls might not like you using share=True. If you're at work on on a work network, I suggest skip this test.

gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(share=True, inbrowser=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://838340c00ca4ba255b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


### Dark mode
Now we will force Gradio to use dark mode as the system theme.

In [13]:
force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""

gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never", js=force_dark_mode).launch()

* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.


Shout has been called with input Xin chào 


### Inputs and outputs space

In [15]:
view = gr.Interface(
    fn=shout,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=[gr.Textbox(label="Response:", lines=8)],
    flagging_mode="never",
    js=force_dark_mode,
)
view.launch()

* Running on local URL:  http://127.0.0.1:7869
* To create a public link, set `share=True` in `launch()`.


In [19]:
# message-gpt
view = gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="Your message: ", lines=6)],
    outputs=[gr.Textbox(label="Response from GPT-4o Mini: ", lines=10)],
    flagging_mode="never",
    js=force_dark_mode
)

view.launch()

* Running on local URL:  http://127.0.0.1:7870
* To create a public link, set `share=True` in `launch()`.


In [20]:
# Get markdown at significant point in GPT response
system_message = "You are a helpful assistant that responds in markdown. Especially, you are a master in Development Engineer."

view = gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="Your message: ")],
    outputs=[gr.Markdown(label="Response from GPT-4o Mini: ")],
    flagging_mode="never",
    js=force_dark_mode,
    )

view.launch()

* Running on local URL:  http://127.0.0.1:7871
* To create a public link, set `share=True` in `launch()`.


## Streaming the response

In [21]:
def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [23]:
view = gr.Interface(
    fn=stream_gpt,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never",
    js=force_dark_mode
)
view.launch()

* Running on local URL:  http://127.0.0.1:7873
* To create a public link, set `share=True` in `launch()`.
